<h1  align="center"><b> MODELO AR </b></h1>

`Objetivo Geral:` Importar a série temporal transformada dos dados pluviométricos do município de São Paulo e realizar a modelagem AR (AutoRegressivo) para previsão.

`Dados:` Os dados foram transformados na pasta [Transformação e Decomposição](../[3]%20Transformação%20e%20Decomposição%20-%20Projeto%20Chuva/) do Projeto Chuva.

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

<h2 align="center"><b> Importando Dados e Escolhendo o Modelo </b></h2>
&emsp;&emsp; O modelo AR (AutoRegressivo) é um modelo de regressão linear que utiliza como variável os valores passados da própria série temporal. Para escolher o melhor modelo utilizaremos o critério de informação de Akaike (AIC) que quanto menor melhor.

`Observação:` O modelo AR(p) é equivalente a ARIMA(p,0,0).



In [81]:
série_chuva = pd.read_csv('../[3] Transformação e Decomposição - Projeto Chuva/Série Transformada - Chuva Mensal.csv', sep = ';', index_col = 0)
série_chuva = pd.Series(série_chuva['Chuva Mensal (mm)'])
série_chuva.index = pd.date_range('1985', periods = len(série_chuva), freq = 'M') # Precisamos disso para identificar a frequência

In [54]:
from statsmodels.tsa.arima.model import ARIMA

# Vendo os valores de AIC para os modelos AR(1) até AR(9). Não convém ir além disso, pois o com o aumento da ordem o tempo de processamento aumenta muito (sem grandes ganhos de AIC)
for i in range(1, 10): 
    modelo_AR = ARIMA(série_chuva, order = (i, 0, 0))
    resultado_AR = modelo_AR.fit()
    print(f'ARIMA({i}, 0, 0): {resultado_AR.aic}')

ARIMA(1, 0, 0): 1577.9031699742663
ARIMA(2, 0, 0): 1578.8492539900853
ARIMA(3, 0, 0): 1548.8664265682196
ARIMA(4, 0, 0): 1515.7737788215243
ARIMA(5, 0, 0): 1468.316854938198
ARIMA(6, 0, 0): 1422.5237584036386
ARIMA(7, 0, 0): 1397.2722987981438
ARIMA(8, 0, 0): 1398.101926577467
ARIMA(9, 0, 0): 1400.0629195660995


In [55]:
modelo_AR = ARIMA(série_chuva, order = (7, 0, 0)) # O modelo ARIMA(7, 0, 0) foi o que apresentou o menor AIC
resultado_AR = modelo_AR.fit() # Treinando o modelo
print(resultado_AR.summary()) # Sumário do modelo

                               SARIMAX Results                                
Dep. Variable:      Chuva Mensal (mm)   No. Observations:                  456
Model:                 ARIMA(7, 0, 0)   Log Likelihood                -689.636
Date:                Sat, 23 Dec 2023   AIC                           1397.272
Time:                        14:58:55   BIC                           1434.375
Sample:                    01-31-1985   HQIC                          1411.888
                         - 12-31-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.7755      0.035    136.216      0.000       4.707       4.844
ar.L1          0.1489      0.043      3.423      0.001       0.064       0.234
ar.L2          0.0823      0.049      1.673      0.0

<h2 align="center"><b> Análise de Resíduos </b></h2>

In [ ]:
resíduo = resultado_AR.resid
resíduo.plot() # Plotando o resíduo
plt.show()

![AR_Residuos](./Gráficos/AR_Residuos.png)

### `Normalidade dos Resíduos:`

In [57]:
import scipy.stats as stats
import seaborn as sns

In [ ]:
stats.probplot(resíduo, dist = 'norm', plot = plt)
plt.title('Normal QQ Plot - Resíduos')
plt.show()

sns.histplot(resíduo, kde = True)
plt.title('Histograma - Resíduos')
plt.show()

![Normal QQ Plot - Resíduos](./Gráficos/Normal%20QQ%20Plot%20-%20Resíduos.png)
![Histograma - Resíduos](./Gráficos/Histograma%20-%20Resíduos.png)

In [59]:
def teste_shapiro(série):
    e, p = stats.shapiro(série)
    print(f'Estatística de Teste = {e}')
    print(f'p-valor = {p}')
    print(f'Resultado: {"É Normal" if (p > 0.05) else "Não é Normal"}')

teste_shapiro(resíduo)

Estatística de Teste = 0.9881777763366699
p-valor = 0.000947304186411202
Resultado: Não é Normal


&emsp;&emsp; Por mais que não seja perfeito, o modelo melhorou bastante em relação ao último teste de normalidade cujo p-valor foi de 7.994796033017337e-05.

### `Autocorrelação dos Resíduos:`

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(resíduo, lags = 20)
plt.title('Função de Autocorrelação - Resíduos')
plt.show()

plot_pacf(resíduo, lags = 20)
plt.title('Função de Autocorrelação Parcial - Resíduos')
plt.show()

![Função de Autocorrelação - Resíduos](./Gráficos/Função%20de%20Autocorrelação%20-%20Resíduos.png)
![Função de Autocorrelação Parcial - Resíduos](./Gráficos/Função%20de%20Autocorrelação%20Parcial%20-%20Resíduos.png)

&emsp;&emsp; As funções de autocorrelação mostram que os resíduos não possuem correlação com os valores passados, o que é um ótimo sinal.

<h2 align="center"><b> Previsão </b></h2>

In [70]:
tamanho_série = len(série_chuva) # Tamanho da série
previsão = resultado_AR.predict(start = tamanho_série, end = tamanho_série + 11) # Previsão para os próximos 12 meses

In [62]:
previsão2 = resultado_AR.forecast(steps = 12) # Método alternativo para previsão

In [ ]:
plt.plot(série_chuva, label = 'Série Original')
plt.plot(série_chuva - resíduo, label = 'Resíduo')
plt.plot(previsão, label = 'Previsão')
plt.title('Previsão - Modelo AR(7)')
plt.legend(loc = 'best')
plt.show()

![Previsão - Modelo AR(7)](./Gráficos/Previsão%20-%20Modelo%20AR(7).png)

&emsp;&emsp; Finalmente chegamos na previsão, mas esses valores passaram por uma transformação por raiz cúbica, então precisamos elevar os valores ao cubo para obter a previsão real.

<h2 align="center"><b> Finalização </b></h2>

In [90]:
previsão_final = previsão ** 3
previsão_final.name = 'Chuva Mensal (mm)'
previsão_final.to_csv('Previsão AR - Chuva Mensal.csv', sep = ';', header = True) # Salvando a previsão em um arquivo csv
display(previsão_final)

2023-01-31    244.748757
2023-02-28    212.566599
2023-03-31    134.929431
2023-04-30     84.136194
2023-05-31     63.237087
2023-06-30     49.195622
2023-07-31     49.770542
2023-08-31     59.228329
2023-09-30     86.840680
2023-10-31    127.146460
2023-11-30    164.883845
2023-12-31    185.861708
Freq: M, Name: Chuva Mensal (mm), dtype: float64

In [89]:
Erro_Quadrático_Médio = (previsão_final['2023-01-31'] - 377.60)**2
Erro_Quadrático_Médio += (previsão_final['2023-02-28'] - 452.00)**2
Erro_Quadrático_Médio += (previsão_final['2023-03-31'] - 138.20)**2
Erro_Quadrático_Médio += (previsão_final['2023-04-30 '] - 165.70)**2
Erro_Quadrático_Médio += (previsão_final['2023-05-31'] - 43.30)**2
Erro_Quadrático_Médio += (previsão_final['2023-06-30'] - 85.20)**2
Erro_Quadrático_Médio += (previsão_final['2023-07-31'] - 15.00)**2
Erro_Quadrático_Médio = Erro_Quadrático_Médio / 7

print(f'Erro Quadrático Médio = {Erro_Quadrático_Médio:.2f}') # Erro Quadrático Médio

Erro Quadrático Médio = 12077.71
